In [1]:
import os

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as krs_image
from sklearn.utils import class_weight
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Useful methods

In [4]:
def create_model(model, high, width):
    
    if model == "VGG16":
        input_shape = (high, width, 3)
        model = tf.keras.applications.VGG16(weights="imagenet", include_top=False, input_shape = input_shape)
        
        #transfer learning
        for layer in model.layers:
            layer.trainable = False
        
        #Adiciono las capas full conected
        x = model.output
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(3, activation= 'softmax', name='predictions')(x)
        
        #Creating my own model
        my_model = Model(inputs = model.input, outputs = x)

    else:
        print("please check model name")
    
    return my_model

In [5]:
def create_generator(train_df, test_df):
    
    #datagenerator using 25% for validation
    datagen = ImageDataGenerator(rescale=1./255., validation_split=0.25)

    train_generator= datagen.flow_from_dataframe(dataframe=train_df,
                                                x_col="path",
                                                y_col="label",
                                                subset="training",
                                                batch_size=8,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=(112, 112))

    valid_generator= datagen.flow_from_dataframe(dataframe=train_df,
                                                x_col="path",
                                                y_col="label",
                                                subset="validation",
                                                batch_size=8,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=(112, 112))

    test_datagen = ImageDataGenerator(rescale=1./255.)

    test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                                    x_col="path",
                                                    y_col="label",
                                                    batch_size=1,
                                                    seed=42,
                                                    shuffle=False,
                                                    class_mode="categorical",
                                                    target_size=(112, 112))
    
    return train_generator, valid_generator, test_generator

In [6]:
def compileModel(model, learning_rate):
    model.compile(optimizer= Adam(lr = learning_rate), loss='categorical_crossentropy', metrics=['acc'])

In [7]:
def train_model(model, high, width, learning_rate, epochs, train_df, test_df, light, fold, 
                experiment, batch_size=8):
        
    train_generator, valid_generator, test_generator = create_generator(train_df, test_df)
    
    ##get model imagenet pretrained
    #cust_model = create_model(model, high, width)
    #print("imagenet model created!")
    
    #get finetuned model
    if (str.upper(experiment)=="NBI AUGMENTED"):
        path = '../models/weights(3clasesNBI)/refinamFALSEVGG163CLASESNBIvideo' + str(fold) + '.h5'
    else:
        print("reading from WL pretrain model...")
        path = '../models/weights(3clasesWL)/refinamFALSEVGG163CLASESWLvideo' + str(fold) + '.h5'
    cust_model = load_model(path, compile = False)
    
    
    save_path = '../models/checkPoints/' + model + 'fold' + str(fold) + '.h5'
    save_csv = '../csvLogger/VGG16' + experiment + '.csv'
    callback_list = [tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', mode='min'),
                    
                    tf.keras.callbacks.ModelCheckpoint(filepath=save_path,
                                                      monitor = 'val_acc',
                                                      verbose=1,
                                                      save_best_only=True,
                                                      mode = 'max',
                                                      save_weights_only=False,
                                                      save_freq='epoch'),
                    tf.keras.callbacks.CSVLogger(filename=save_csv, separator=',', append=False),
                    ]
    
    if(str.upper(light)=="NBI"):
        class_weight = {0:1, 1:3, 2:3}
        print("pesos usados: adenoma",1,"hiperplastic: ", 3,"serrated: ",3)
    else:
        class_weight = {0:1, 1:3, 2:4}
        print("pesos usados: adenoma",1,"hiperplastic: ", 3,"serrated: ",4)
        
    compileModel(cust_model, learning_rate)
    print("modelo compilado!")
    
    history = cust_model.fit(train_generator, steps_per_epoch= 100, epochs= epochs, verbose=1,
                             callbacks=callback_list, validation_data=valid_generator,
                             validation_steps= 100, class_weight=class_weight, shuffle=False,
                             initial_epoch=0)
    
    return cust_model, valid_generator, test_generator

In [8]:
def eval_model(model, valid_generator):
    print("=============model evaluating with validation data...===================================")
    val_loss, val_acc = model.evaluate_generator(generator = valid_generator, steps=10)
    print('validation accuracy:', val_acc)
    print('validation loss:', val_loss)
    return val_loss, val_acc

In [9]:
def test_model(model, test_df, test_generator, num_fold, experiment):
    print("=============testing model...===================================")
    for i in range(10,110,10):
        n = int(len(test_df)*(i/100))
        print("amount of frames ", n)
        test_generator.reset()
        pred = model.predict(test_generator, steps = n, verbose=1)
        predicted_class_indices=np.argmax(pred,axis=1)    

        print("========== amount of elements over prediction =========")
        txtfile.write("==========percentaje of elements over testing data:" + str(i/100) + "=========\n")
        lista = predicted_class_indices.tolist()        
        print("predicted", lista.count(0), "as adenoma")
        print("predicted", lista.count(1), "as hiperplastico")
        print("predicted", lista.count(2), "as serrated")
        txtfile.write("predicted " + str(lista.count(0)) + "as adenoma\n")
        txtfile.write("predicted " + str(lista.count(1)) + "as hiperplastico\n")
        txtfile.write("predicted " + str(lista.count(2)) + "as serrated\n") 
        
       
    #defining path according to experiment
    if (str.upper(experiment)=="NBI AUGMENTED"):
        path = '../models/weights(scratch3clasesNBIaugmented)/refinamFALSEVGG163CLASESNBIvideo' + str(num_fold) + '.h5'
        print("guardando en: weights(scratch3clasesNBIaugmented)")
    else:
        path = '../models/weights(preTrain3clasesWLaugmented)/refinamFALSEVGG163CLASESWLvideo' + str(num_fold) + '.h5'
        print("guardando en: weights(preTrain3clasesWLaugmented)")
        
    model.save(path)
    print("model saved!")

# Main

In [10]:
train_df = pd.read_csv('../data/WL_augmented/datasetWLaugmented.csv', header=None)
train_df.columns = ['path', 'label']
print(train_df.shape)
train_df.head()

(52795, 2)


,path,label
0,../data/WL_augmented//adenoma_WL/video_27/aden...,adenoma
1,../data/WL_augmented//adenoma_WL/video_27/aden...,adenoma
2,../data/WL_augmented//adenoma_WL/video_27/aden...,adenoma
3,../data/WL_augmented//adenoma_WL/video_27/aden...,adenoma
4,../data/WL_augmented//adenoma_WL/video_27/aden...,adenoma


In [ ]:
acc, loss = [], []
light = "WL"
model = "VGG16"
high, width = 112, 112
experiment="WL AUGMENTED"
#adenoma
txtfile = open('refinamFALSEVGG16'+ experiment + 'Adenoma.txt', 'w+')
txtfile.write('==========predicting adenoma class ==========\n')
txtfile.write('========== 3 CLASES con luz blanca==========\n') 
txtfile.write('==========Network used: VGG16 ==========\n')
txtfile.write('==========Without refinament: ==========\n')
clase = "adenoma_" + light + "/"
cant = 40
for i in range(1,cant+1): 
    print("=================================kfold #", i,"===========================================")  
    print("==== testing with adenoma class ====")
    txtfile.write("======Kfold:"+str(i)+"======\n")        
    current_test_df = train_df[train_df['path'].str.contains(clase+'video_'+str(i)+'/')]
    current_train_df = train_df[~train_df["path"].str.contains(clase+'video_'+str(i)+'/')]
    #training data
    model_trained, valid_generator, test_generator = train_model(model, high, width, 1e-4, 100, 
                                                                current_train_df, current_test_df, light, i, 
                                                                experiment)
    #validate data
    vlr_loss, vlr_acc = eval_model(model_trained, valid_generator)
    txtfile.write("=========behavior over validation data:=========\n" ) 
    txtfile.write("validation accuracy " + str(vlr_acc) + "\n")
    txtfile.write("validation loss " + str(vlr_loss) + "\n")
    acc.append(vlr_acc)
    loss.append(vlr_loss)
    #testing data
    test_model(model_trained, current_test_df, test_generator, i, experiment)
txtfile.close()

#hiperplastic
txtfile = open('refinamFALSEVGG16' + experiment + 'Hyperplastic.txt', 'w+')
txtfile.write('==========predicting over hiperplastic class ==========\n')
txtfile.write('========== 3 CLASES con luz blanca ==========\n')  
txtfile.write('========== Network used: VGG16 ==========\n')
txtfile.write('========== Without refinament: ==========\n')
clase = "hiperplastic_" + light + "/"
cant = 21
for i in range(1,cant+1):
    print("====================================kfold #", 40+i,"==================================================") 
    print("====testing with hiperplastic class====")
    txtfile.write("======Kfold:"+str(40+i)+"======\n")
    num_fold = 40+i
    current_test_df = train_df[train_df['path'].str.contains(clase+'video_'+str(i)+'/')]
    current_train_df = train_df[~train_df["path"].str.contains(clase+'video_'+str(i)+'/')]
    #mando a entrenar
    model_trained, valid_generator, test_generator = train_model(model, high, width, 1e-4, 100,
                                                                           current_train_df, current_test_df,
                                                                           light, num_fold, experiment)
    #validate data
    vlr_loss, vlr_acc = eval_model(model_trained, valid_generator)
    txtfile.write("=========metricas del comportamiento en el video completo:=========\n" ) 
    txtfile.write("validation accuracy " + str(vlr_acc) + "\n")
    txtfile.write("validation loss " + str(vlr_loss) + "\n")
    acc.append(vlr_acc)
    loss.append(vlr_loss)
    #testing data
    test_model(model_trained, current_test_df, test_generator, num_fold, experiment)
txtfile.close()

#probando con los de serrated
txtfile = open('refinamFALSEVGG16' + experiment + 'Serrated.txt', 'w+')
txtfile.write('==========predicting over serrated class ==========\n')
txtfile.write('========== 3 CLASES con luz blanca ==========\n')  
txtfile.write('========== Network used: VGG16 ==========\n')
txtfile.write('========== Without refinament: ==========\n')
clase = "serrated_" + light + "/"
cant = 15
for i in range(1,cant+1):  
    print("====================================kfold #", 61+i,"===========================================")  
    print("==== testing over serrated class ====")   
    txtfile.write("======Kfold:"+str(61+i)+"======\n") 
    num_fold = 61+i
    current_test_df = train_df[train_df['path'].str.contains(clase+'video_'+str(i)+'/')]
    current_train_df = train_df[~train_df["path"].str.contains(clase+'video_'+str(i)+'/')]
    #mando a entrenar
    model_trained, valid_generator, test_generator = train_model(model, high, width, 1e-4,
                                                                 100, current_train_df,
                                                                 current_test_df, light, num_fold, experiment)
    #validate data
    vlr_loss, vlr_acc = eval_model(model_trained, valid_generator)
    txtfile.write("=========metricas del comportamiento en el video completo:=========\n" ) 
    txtfile.write("validation accuracy " + str(vlr_acc) + "\n")
    txtfile.write("validation loss " + str(vlr_loss) + "\n")
    acc.append(vlr_acc)
    loss.append(vlr_loss)
    #testing data
    test_model(model_trained, current_test_df, test_generator, num_fold, experiment)

npa_acc = np.asarray(acc)
npa_loss = np.asarray(loss)

print("========================model performance...==================")
txtfile.write("========================model performance...==================\n")
npa_acc = np.asarray(acc)
npa_loss = np.asarray(loss)

print("average accuracy: ", np.mean(npa_acc))
print("average loss: ", np.mean(npa_loss))

txtfile.write("average accuracy : " + str(np.mean(npa_acc)) + "\n")
txtfile.write("average loss : " + str(np.mean(npa_loss)) + "\n")

txtfile.close()  

=================================kfold # 1 ===========================================
==== testing with adenoma class ====
Found 39458 validated image filenames belonging to 3 classes.
Found 13152 validated image filenames belonging to 3 classes.
Found 185 validated image filenames belonging to 1 classes.
reading from WL pretrain model...
pesos usados: adenoma 1 hiperplastic:  3 serrated:  4
modelo compilado!


/usr/local/lib/python3.6/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
100/100 [==============================] - 19s 144ms/step - loss: 0.2518 - acc: 0.9550 - val_loss: 2.7409 - val_acc: 0.4775

Epoch 00001: val_acc improved from -inf to 0.47750, saving model to ../models/checkPoints/VGG16fold1.h5
Epoch 2/100
100/100 [==============================] - 40s 397ms/step - loss: 0.1950 - acc: 0.9588 - val_loss: 3.2353 - val_acc: 0.3363

Epoch 00002: val_acc did not improve from 0.47750
Epoch 3/100
100/100 [==============================] - 18s 176ms/step - loss: 0.3744 - acc: 0.9375 - val_loss: 3.3464 - val_acc: 0.4013

Epoch 00003: val_acc did not improve from 0.47750
Epoch 4/100
100/100 [==============================] - 18s 176ms/step - loss: 0.2736 - acc: 0.9500 - val_loss: 2.0658 - val_acc: 0.4837

Epoch 00004: val_acc improved from 0.47750 to 0.48375, saving model to ../models/checkPoints/VGG16fold1.h5
Epoch 5/100
100/100 [==============================] - 18s 181ms/step - loss: 0.3262 - acc: 0.9388 - val_loss: 0.9763 - val_acc: 0.7475

Epoc

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


validation accuracy: 0.6000000238418579
validation loss: 1.6684982776641846
=============testing model...===================================
amount of frames  18
18/18 [==============================] - 1s 35ms/step
========== amount of elements over prediction =========
predicted 0 as adenoma
predicted 0 as hiperplastico
predicted 18 as serrated
amount of frames  37
37/37 [==============================] - 1s 30ms/step
========== amount of elements over prediction =========
predicted 0 as adenoma
predicted 0 as hiperplastico
predicted 37 as serrated
amount of frames  55
55/55 [==============================] - 1s 25ms/step
========== amount of elements over prediction =========
predicted 0 as adenoma
predicted 0 as hiperplastico
predicted 55 as serrated
amount of frames  74
74/74 [==============================] - 2s 23ms/step
========== amount of elements over prediction =========
predicted 0 as adenoma
predicted 0 as hiperplastico
predicted 74 as serrated
amount of frames  92
92/92 

...

100/100 [==============================] - 19s 182ms/step - loss: 0.2847 - acc: 0.9525 - val_loss: 2.4592 - val_acc: 0.3850

Epoch 00001: val_acc improved from -inf to 0.38500, saving model to ../models/checkPoints/VGG16fold25.h5
Epoch 2/100
100/100 [==============================] - 19s 186ms/step - loss: 0.2592 - acc: 0.9425 - val_loss: 4.1694 - val_acc: 0.2000

Epoch 00002: val_acc did not improve from 0.38500
Epoch 3/100
100/100 [==============================] - 18s 182ms/step - loss: 0.1601 - acc: 0.9737 - val_loss: 1.8201 - val_acc: 0.6325

Epoch 00003: val_acc improved from 0.38500 to 0.63250, saving model to ../models/checkPoints/VGG16fold25.h5
Epoch 4/100
100/100 [==============================] - 17s 174ms/step - loss: 0.1257 - acc: 0.9725 - val_loss: 2.3185 - val_acc: 0.5437

Epoch 00004: val_acc did not improve from 0.63250
Epoch 5/100
100/100 [==============================] - 17s 175ms/step - loss: 0.1419 - acc: 0.9750 - val_loss: 2.0509 - val_acc: 0.5238

Epoch 00005: v

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


validation accuracy: 0.4375
validation loss: 2.7785096168518066
=============testing model...===================================
amount of frames  193
193/193 [==============================] - 3s 16ms/step
========== amount of elements over prediction =========
predicted 64 as adenoma
predicted 26 as hiperplastico
predicted 103 as serrated
amount of frames  387
387/387 [==============================] - 6s 15ms/step
========== amount of elements over prediction =========
predicted 122 as adenoma
predicted 52 as hiperplastico
predicted 213 as serrated
amount of frames  580
580/580 [==============================] - 9s 16ms/step
========== amount of elements over prediction =========
predicted 127 as adenoma
predicted 107 as hiperplastico
predicted 346 as serrated
amount of frames  774
774/774 [==============================] - 12s 16ms/step
========== amount of elements over prediction =========
predicted 178 as adenoma
predicted 232 as hiperplastico
predicted 364 as serrated
amount of

/usr/local/lib/python3.6/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
100/100 [==============================] - 19s 182ms/step - loss: 0.2163 - acc: 0.9625 - val_loss: 1.7208 - val_acc: 0.5512

Epoch 00001: val_acc improved from -inf to 0.55125, saving model to ../models/checkPoints/VGG16fold26.h5
Epoch 2/100
100/100 [==============================] - 18s 181ms/step - loss: 0.1880 - acc: 0.9575 - val_loss: 4.2605 - val_acc: 0.2450

Epoch 00002: val_acc did not improve from 0.55125
Epoch 3/100
100/100 [==============================] - 18s 179ms/step - loss: 0.2912 - acc: 0.9500 - val_loss: 2.7058 - val_acc: 0.5163

Epoch 00003: val_acc did not improve from 0.55125
Epoch 4/100
100/100 [==============================] - 18s 177ms/step - loss: 0.2224 - acc: 0.9588 - val_loss: 1.6288 - val_acc: 0.6263

Epoch 00004: val_acc improved from 0.55125 to 0.62625, saving model to ../models/checkPoints/VGG16fold26.h5
Epoch 5/100
100/100 [==============================] - 18s 181ms/step - loss: 0.0704 - acc: 0.9875 - val_loss: 2.2398 - val_acc: 0.5625

Ep